In [2]:
%reset

In [4]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import datetime
import math
import gc

In [5]:
print("loading")
data_path='/Users/yzh/Desktop/sales/sale/'
test= pd.read_csv( data_path+'test.csv', 
                   dtype={'item_nbr':'int32', 'store_nbr':'int8'},parse_dates=['date'])
train= pd.read_csv( data_path+'train.csv', dtype={'item_nbr':'int32', 'store_nbr':'int8','unit_sales':'float32'},
                    parse_dates=['date'], usecols=[1,2,3,4,5])#Skip dates before 2016-08-01 or 2016-01-01

loading


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
train=train[ ( (datetime.date(2015,6,1) <= train['date']) &  (train['date'] < datetime.date(2015,9,1)) )
             |( (datetime.date(2016,6,1) <= train['date']) &  (train['date'] < datetime.date(2016,9,1)) )
             | ( datetime.date(2017,6,1) <= train['date']  ) ]

train["on_shelf"]=True
test["on_shelf"]=True

In [7]:
train.onpromotion.fillna(-1,inplace=True)  # -1 for na
store_item=train[["store_nbr","item_nbr"]] 
store_item.drop_duplicates(inplace=True)
store_item["exist"]=1 # to store all existing store-item pair


u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
).reset_index()
train.unit_sales.fillna(0,inplace=True)   # 0 for missing sales data
train.onpromotion.fillna(-2,inplace=True)  # -2 for missing promotion data
#train.on_shelf.fillna(False,inplace=True)

train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
del u_dates,u_stores,u_items

train= train.merge(store_item, on=["store_nbr","item_nbr"],how="left")
train=train[train.exist==1]
train.drop(["exist"],axis=1, inplace=True) # delete non_existing store_item pairs

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [16]:
store_item=train[["store_nbr","item_nbr"]]
store_item.drop_duplicates(inplace=True)
store_item=store_item.groupby(["store_nbr"],as_index=False).agg({"item_nbr":{"store_count":"count"}})
store_item.columns=store_item.columns.droplevel(level=0)
store_item.columns.values[0]="store_nbr"
train=train.merge(store_item,on=['store_nbr'],how="left")
test=test.merge(store_item,on=['store_nbr'],how="left")

store_item=train[["store_nbr","item_nbr"]]
store_item.drop_duplicates(inplace=True)
store_item=store_item.groupby(["item_nbr"],as_index=False).agg({"store_nbr":{"item_count":"count"}})
store_item.columns=store_item.columns.droplevel(level=0)
store_item.columns.values[0]="item_nbr"
train=train.merge(store_item,on=['item_nbr'],how="left")
test=test.merge(store_item,on=['item_nbr'],how="left")

In [36]:
gc.collect()

584

average_all=train.groupby(["store_nbr","item_nbr"],as_index=False).agg({"unit_sales":{"average_all":"mean"}})
average_all.columns=average_all.columns.droplevel(level=0)
average_all.columns.values[0]="store_nbr"
average_all.columns.values[1]="item_nbr"
train=train.merge(average_all, on=["store_nbr","item_nbr"], how="left")
test=test.merge(average_all, on=["store_nbr","item_nbr"], how="left")

In [ ]:
average_1=train[(datetime.date(2015,6,1) <= train['date']) &  (train['date'] < datetime.date(2015,9,1))].groupby(["store_nbr","item_nbr"]
                            ,as_index=False).agg({"unit_sales":{"average_1":"mean"}})
average_1.columns=average_1.columns.droplevel(level=0)
average_1.columns.values[0]="store_nbr"
average_1.columns.values[1]="item_nbr"
train=train.merge(average_1, on=["store_nbr","item_nbr"], how="left")
test=test.merge(average_1, on=["store_nbr","item_nbr"], how="left")

average_2=train[(datetime.date(2016,6,1) <= train['date']) &  (train['date'] < datetime.date(2016,9,1))].groupby(["store_nbr","item_nbr"]
                        ,as_index=False).agg({"unit_sales":{"average_2":"mean"}})
average_2.columns=average_2.columns.droplevel(level=0)
average_2.columns.values[0]="store_nbr"
average_2.columns.values[1]="item_nbr"
train=train.merge(average_2, on=["store_nbr","item_nbr"], how="left")
test=test.merge(average_2, on=["store_nbr","item_nbr"], how="left")

#del average_all,average_1,average_2,store_item
average_4=train[datetime.date(2017,6,1) <= train['date']].groupby(["store_nbr","item_nbr"]
                        ,as_index=False).agg({"unit_sales":{"average_4":"mean"}})
average_4.columns=average_4.columns.droplevel(level=0)
average_4.columns.values[0]="store_nbr"
average_4.columns.values[1]="item_nbr"
train=train.merge(average_4, on=["store_nbr","item_nbr"], how="left")
test=test.merge(average_4, on=["store_nbr","item_nbr"], how="left")

average_hasSale=train[train["unit_sales"]>0].groupby(["store_nbr","item_nbr"],as_index=False).agg({"unit_sales":{"average_hasSale":"mean"}})
average_hasSale.columns=average_hasSale.columns.droplevel(level=0)
average_hasSale.columns.values[0]="store_nbr"
average_hasSale.columns.values[1]="item_nbr"
train=train.merge(average_hasSale, on=["store_nbr","item_nbr"], how="left")
test=test.merge(average_hasSale, on=["store_nbr","item_nbr"], how="left")

print(train[["average_1","average_2","average_4","unit_sales"]].isnull().sum())

summary_weekday=train.groupby(["store_nbr","item_nbr"],as_index=False).agg({"unit_sales":{"average_all":
               "mean"}})
average_all.columns=average_all.columns.droplevel(level=0)
average_all.columns.values[0]="store_nbr"
average_all.columns.values[1]="item_nbr"
train=train.merge(average_all, on=["store_nbr","item_nbr"], how="left")
test=test.merge(average_all, on=["store_nbr","item_nbr"], how="left")

In [338]:
gc.collect()

245

In [ ]:
items=pd.read_csv(data_path+'items.csv',dtype={'item_nbr':'int32','family':'category','class':'category','perishable':'category'})
test=test.merge(items,how="left",on="item_nbr")
train=train.merge(items,how='left',on="item_nbr")

stores=pd.read_csv(data_path+'stores.csv',dtype={'store_nbr':'int8','city':'category','state':'category','type':'category','cluster':'category'})
test=test.merge(stores,how="left",on="store_nbr")
train=train.merge(stores,how="left",on="store_nbr")

oil=pd.read_csv(data_path+'oil.csv',dtype={'dcoilwtico':"float32"},parse_dates=['date'])
test=test.merge(oil,how="left",on="date")
train=train.merge(oil,how="left",on="date")

test['year'] = test['date'].dt.year.astype(np.int16)
test['month'] = test['date'].dt.month.astype(np.int8)
test['day'] = test['date'].dt.day.astype(np.int8)
train['year'] = train['date'].dt.year.astype(np.int16)
train['month'] = train['date'].dt.month.astype(np.int8)
train['day'] = train['date'].dt.day.astype(np.int8)

#ini_date=train.at[0,'date']
#test['time_diff']=( (test['date']-ini_date).dt.total_seconds() )/(3600*24)
#train['time_diff']=((train['date']-ini_date).dt.total_seconds() )/(3600*24)

test['weekday']=test['date'].dt.weekday.astype(np.int8)
train['weekday']=train['date'].dt.weekday.astype(np.int8)

holidays=pd.read_csv(data_path+'holidays_events.csv',dtype={'type':'category','locale':'category','locale_name':'category','description':'category','transferred':'category'},
parse_dates=['date'])
holidays.columns=['date','holi_type','locale','locale_name','description','transferred']
train=train.merge(holidays,how="left",on="date")
test=test.merge(holidays,how="left",on="date")

del items, stores, oil, holidays

In [ ]:
summary_summer=train.groupby(["year","store_nbr","item_nbr"],as_index=False).agg({"unit_sales":{"summary_summer":"mean"}})
summary_summer.columns=summary_summer.columns.droplevel(level=0)
summary_summer.columns.values[0]="year"
summary_summer.columns.values[1]="store_nbr"
summary_summer.columns.values[2]="item_nbr"
train=train.merge(summary_summer, on=["store_nbr","item_nbr","year"], how="left")
test=test.merge(summary_summer, on=["store_nbr","item_nbr","year"], how="left")

summary_HasSaleSummer=train[train["unit_sales"]>0].groupby(["year","store_nbr","item_nbr"]
            ,as_index=False).agg({"unit_sales":{"summary_HasSaleSummer":"mean"}})
summary_HasSaleSummer.columns=summary_HasSaleSummer.columns.droplevel(level=0)
summary_HasSaleSummer.columns.values[0]="year"
summary_HasSaleSummer.columns.values[1]="store_nbr"
summary_HasSaleSummer.columns.values[2]="item_nbr"
train=train.merge(summary_HasSaleSummer, on=["store_nbr","item_nbr","year"], how="left")
test=test.merge(summary_HasSaleSummer, on=["store_nbr","item_nbr","year"], how="left")

summary_item_mean=train.groupby(["year","item_nbr"]
            ,as_index=False).agg({"unit_sales":{"summary_item_mean":"mean"}})
summary_item_mean.columns=summary_item_mean.columns.droplevel(level=0)
summary_item_mean.columns.values[0]="year"
summary_item_mean.columns.values[1]="item_nbr"
train=train.merge(summary_item_mean, on=["item_nbr","year"], how="left")
test=test.merge(summary_item_mean, on=["item_nbr","year"], how="left")
del summary_item_mean
print("done")

In [8]:
train["unit_sales"]=train["unit_sales"].astype(np.float16)

In [9]:
ini_date=train.at[0,'date']
test['time_diff']=( (test['date']-ini_date).dt.total_seconds() )/(3600*24)
train['time_diff']=((train['date']-ini_date).dt.total_seconds() )/(3600*24)
test['time_diff']=test["time_diff"].astype(np.int16)
train['time_diff']=train["time_diff"].astype(np.int16)

train['item_nbr']=train['item_nbr'].astype(np.int32)
train['store_nbr']=train['store_nbr'].astype(np.int8)
test['item_nbr']=test['item_nbr'].astype(np.int32)
test['store_nbr']=test['store_nbr'].astype(np.int8)

train_ar=train[['store_nbr','item_nbr','time_diff','unit_sales']]
train_ar.set_index(['store_nbr', 'item_nbr','time_diff'], inplace=True)
gc.collect()
print("to_dict")
train_dict=train_ar.to_dict(orient='index')
print("done")
del train_ar

to_dict


done


In [11]:
train.dtypes

date           datetime64[ns]
store_nbr                int8
item_nbr                int32
unit_sales            float16
onpromotion            object
time_diff               int16
dtype: object

In [12]:
%load_ext Cython

In [13]:
%%cython
cimport numpy as np
import numpy as np
def  get_past_c(  store,   item,   time, train_dict):
     try:
       return (train_dict[(store, item, time)])['unit_sales']
     except KeyError:
       return -1
cpdef np.ndarray[double] apply_integrate(np.ndarray col_a, np.ndarray col_b, np.ndarray col_N, train_dict):
#     assert (col_a.dtype == np.int and col_b.dtype == np.int and col_N.dtype == np.int)
     cdef Py_ssize_t i, n = len(col_N)
#     assert (len(col_a) == len(col_b) == n)
     cdef np.ndarray[double] res = np.empty(n)
     for i in range(len(col_a)):
         res[i] = get_past_c(col_a[i], col_b[i], col_N[i], train_dict)
     return res

In [15]:
for day in [7,14,21,28,35,42,49]: #test, train's keys had better be int type
    past=apply_integrate(test['store_nbr'].values, test['item_nbr'].values, test['time_diff'].values-day,train_dict)
    past=np.where(past!=-1,past,np.nan)
    test[ ("past_"+str(day) )] = past
    test[ ("past_"+str(day) )]= test[ ("past_"+str(day) )].astype(np.float16)
    
    past=apply_integrate(train['store_nbr'].values, train['item_nbr'].values, train['time_diff'].values-day,train_dict)
    past=np.where(past!=-1,past,np.nan)
    train[ ("past_"+str(day) )] = past
    train[ ("past_"+str(day) )]= train[ ("past_"+str(day) )].astype(np.float16)
    print("one recored")

one recored


one recored


one recored


one recored


one recored


one recored


one recored


In [5]:
train.dtypes

NameError: name 'train' is not defined

In [15]:
test["mean_7_y"]=test [["past_14","past_21","past_28","past_35"]].mean(axis=1)
train["mean_7_y"]=train[["past_14","past_21","past_28","past_35"]].mean(axis=1)
test["mean_7_z"]=test [["past_14","past_21","past_28","past_35","past_42","past_49"]].mean(axis=1)
train["mean_7_z"]=train[["past_14","past_21","past_28","past_35","past_42","past_49"]].mean(axis=1)

summary_weekday_all=train.groupby(["weekday","store_nbr","item_nbr"],as_index=False).agg({"unit_sales":{"summary_weekday_all":"mean"}})
summary_weekday_all.columns=summary_weekday_all.columns.droplevel(level=0)
summary_weekday_all.columns.values[0]="weekday"
summary_weekday_all.columns.values[1]="store_nbr"
summary_weekday_all.columns.values[2]="item_nbr"
train=train.merge(summary_weekday_all, on=["store_nbr","item_nbr","weekday"], how="left")
test=test.merge(summary_weekday_all, on=["store_nbr","item_nbr","weekday"], how="left")

train["weekday_ratio"]=train["summary_weekday_all"]/train["average_all"]
test["weekday_ratio"]=test["summary_weekday_all"]/test["average_all"]

for i in range(0,100) #test.shape[0]
    train[(train.time_diff>=test.at[i,'time_diff']-29) and (train.time_diff<=test.at[i,'time_diff']-16) and (train.store_nbr==test.at[i,'store_nbr'])
               and (train.item_nbr==test.at[i,'item_nbr'])]

train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train.reset_index(inplace=True)

train.set_index(['time_diff', 'store_nbr', 'item_nbr'],inplace=True)

train.at[ slice(test.at[i,'time_diff']-50, test.at[i,'store_nbr'], test.at[i,'item_nbr']),'unit_sales']# store_nbr[test.at[i,'store_nbr']], item_nbr[ test.at[i,'item_nbr'] ] ]

train[(train.time_diff>=test.at[i,'time_diff']-29) & (train.time_diff<=test.at[i,'time_diff']-16) & (train.store_nbr==test.at[i,'store_nbr'])
           & (train.item_nbr==test.at[i,'item_nbr'])]['unit_sales'].mean()

In [ ]:
train['item_nbr']=train['item_nbr'].astype(np.int32)
train['store_nbr']=train['store_nbr'].astype(np.int8)
test['item_nbr']=test['item_nbr'].astype(np.int32)
test['store_nbr']=test['store_nbr'].astype(np.int8)

for col in ['store_nbr','item_nbr','onpromotion','weekday','perishable']:
    test[col] = test[col].astype('category')
for col in ['store_nbr','item_nbr','onpromotion','weekday','perishable']:
    train[col] = train[col].astype('category')

In [ ]:
for col in train.columns:
    if train[col].dtype == np.float32 or train[col].dtype == np.float64:
        train[col] = train[col].astype(np.float16)
for col in test.columns:
    if test[col].dtype == np.float32 or test[col].dtype == np.float64:
        test[col] = test[col].astype(np.float16)

train["time_diff"]=train["time_diff"].astype(np.int16)
test["time_diff"]=test["time_diff"].astype(np.int16)
train["average_1"]=train["average_1"].astype(np.float16)
test["average_1"]=test["average_1"].astype(np.float16)

train=train[train.unit_sales!=0]

In [318]:
gc.collect()

296

In [ ]:
#train2=train[train.onpromotion!=-2]
def perish(x):
    if x=="1":
        return 1.25
    else:
        return 1
weight_train=train["perishable"].apply(perish).astype(np.float16)

def promo(x):
    if x==True:
        return 1.5
    else:
        return 1
sellmore=test["onpromotion"].apply(promo).astype(np.float16)

def concertNa(x):
    if x==-1:
      return np.nan
    else:
      return x
train2["onpromotion"]=train2["onpromotion"].apply(concertNa).astype("category")

In [ ]:
#x_test=test.drop(['id','date',"onpromotion"],axis=1)
ids = test['id'].values
#train['unit_sales'].values
train_final = lgb.Dataset( train.drop(['unit_sales','date',"onpromotion"],axis=1), 
                        train['unit_sales'].values, weight=weight_train) #np.log1p( np.where(sale>0, sale, 0)

In [200]:
del oil, items, stores, store_item

In [355]:
print("starting")
params = {
    'objective': 'regression_l2',
    'metric': 'l2_root',
    'learning_rate': 0.3,
    'num_leaves': 75,
    'num_rounds': 63,
}
model_f2 = lgb.train(params,
                     train_set=train_final,valid_sets=train_final, verbose_eval=20)

starting


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[20]	training's rmse: 0.554471


[40]	training's rmse: 0.545304


[60]	training's rmse: 0.540081


In [3]:
print("do")

do


In [356]:
gc.collect()

25

In [358]:
p_test_2 = np.expm1(model_f2.predict( test.drop(['id','date',"onpromotion"],axis=1) ) )

In [358]:
p_test_posi=(sellmore* (np.where(p_test_2>0, p_test_2, 0)) ).values

p_test_posi = ( p_test_posi/sellmore ).values

In [312]:
p_test_posi = np.where(p_test_2>0, p_test_2, 0)

In [360]:
subm = pd.DataFrame()
subm['id'] = ids
subm['unit_sales'] = p_test_posi
subm.to_csv('/Users/yzh/Desktop/sales/csv/lgbm_summerData_storeitem_final_WeightEasyPromo.csv.gz', compression = 'gzip', 
            index=False, float_format = '%.5f')

print('Done!')

Done!


In [177]:
print("ddone")

ddone


In [262]:
train[["summary_summer","average_hasSale","unit_sales"]].head(1000)

,summary_summer,average_hasSale,unit_sales
0,1.068617,1.163144,1.098612
1,1.066320,1.341491,0.693147
2,1.563300,1.661572,1.945910
3,2.420338,2.336398,2.564949
4,0.289639,1.199493,0.693147
5,1.082345,1.655754,1.609438
6,1.077014,1.170308,0.693147
7,0.452029,0.942318,2.079442
8,0.606949,1.015103,1.386294
9,0.972852,1.208910,1.098612


In [359]:
gain = model_f2.feature_importance('gain')
ft = pd.DataFrame({'feature':model_f2.feature_name(), 'split':model_f2.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
ft

,feature,gain,split
18,summary_summer,94.720739,367
1,item_nbr,1.446891,1755
0,store_nbr,0.730956,567
17,weekday,0.697269,266
12,dcoilwtico,0.595887,99
16,time_diff,0.420815,412
15,day,0.271008,217
19,summary_HasSaleSummer,0.268421,248
13,year,0.262185,105
6,class,0.219872,187


In [3]:
df = pd.DataFrame({'a': np.random.randn(1000),
 'b': np.random.randn(1000),'N': np.random.randint(100, 1000, (1000)),
 'x': 'x'})

In [6]:
def integrate_f(a,b,n):
  return a+b+n

In [7]:
%timeit df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

65.9 ms ± 15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
p=%prun -r df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

In [18]:
p.print_stats()

         118576 function calls (113567 primitive calls) in 0.157 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     3000    0.020    0.000    0.100    0.000 base.py:2537(get_value)
     3000    0.013    0.000    0.115    0.000 series.py:620(__getitem__)
        1    0.010    0.010    0.150    0.150 {pandas._libs.lib.reduce}
     9027    0.008    0.000    0.018    0.000 {built-in method builtins.getattr}
     3000    0.008    0.000    0.008    0.000 {method 'get_value' of 'pandas._libs.index.IndexEngine' objects}
     3000    0.008    0.000    0.014    0.000 internals.py:4513(get_values)
     6000    0.007    0.000    0.024    0.000 {pandas._libs.lib.values_from_object}
     3000    0.007    0.000    0.020    0.000 base.py:1360(_convert_scalar_indexer)
     8118    0.006    0.000    0.009    0.000 {built-in method builtins.isinstance}
10067/5058    0.006    0.000    0.009    0.000 {built-in method builtins.len}
     1000  